In [1]:
!pip install pymystem3
!pip install pymorphy2
!pip install razdel
!pip install gensim
!pip install nltk
!pip install rusenttokenize
!pip install regex

In [2]:
from razdel import tokenize
import nltk

In [3]:
! pip install pymystem3

In [4]:
from pymystem3 import Mystem

In [5]:
import re
import nltk


# Задание 1

In [31]:
import pymorphy2 as pm
morph = pm.MorphAnalyzer()
text = "Зонды будут запущены в космос в 2029 году и отправятся ко второй точке Лагранжа в системе Солнце"
textTokens = text.split(' ')
textParts = [] 
verbs = []
for i in textTokens:
    pos = morph.parse(i.lower())[0].tag.POS
    textParts.append([i, pos])
    if pos == "VERB":
        verbs.append([i, pos])
print(textParts)
print(verbs)

[['Зонды', 'NOUN'], ['будут', 'VERB'], ['запущены', 'PRTS'], ['в', 'PREP'], ['космос', 'NOUN'], ['в', 'PREP'], ['2029', None], ['году', 'NOUN'], ['и', 'CONJ'], ['отправятся', 'VERB'], ['ко', 'PREP'], ['второй', 'ADJF'], ['точке', 'NOUN'], ['Лагранжа', 'NOUN'], ['в', 'PREP'], ['системе', 'NOUN'], ['Солнце', 'NOUN']]
[['будут', 'VERB'], ['отправятся', 'VERB']]


# Задание 2
На данных предложениях обучите TFIDF векторайзер из sklearn (с дефолтными параметрами). Найдите слово с самым высоким коэффициентом tfidf в первом тексте. Вставьте его в форму.

In [34]:
import requests
r = requests.get(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt'
)
sentences = r.text.split('\n')

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import operator

In [37]:
sentences = [sent for sent in sentences if len(sent) > 10]

vectorizer = TfidfVectorizer()

x = vectorizer.fit_transform(sentences)

feature_index = x[0,:].nonzero()[1]
tfidf_scores = zip(feature_index, [x[0, i] for i in feature_index])
rate = []
for w, s in [(vectorizer.get_feature_names()[i], s) for (i, s) in tfidf_scores]:
    rate.append([w, s])
sortRate  = sorted(rate, key = operator.itemgetter(1,0))
print(sortRate[-1])

['устои', 0.19671514407338786]


# Задание 3
Токенизируйте текст Анны Карениной с помощью библиотеки раздел, привидите все к нижнему регистру. Рассчитайте статистики биграммов и униграммов. Найдите наиболее вероятное продолжение "красный ...". Вставьте наиболее веротяное следующее слово в форму.

In [77]:
import razdel
import gensim
from collections import Counter
from razdel import tokenize

In [88]:
a = []
for sentence in sentences:
    tokens = list(tokenize(sentence))
    a.append([_.text.lower() for _ in tokens])

from nltk import word_tokenize 
from nltk.util import ngrams

bigram_list = []

for i in a:
    bigram = list(ngrams(i, 2))
    bigram_list.append(bigram)
    
bigram_list

bigram_list_flat = []

for i in bigram_list:
    for j in i:
        if j[0] == 'красный':
            bigram_list_flat.append(j)

Counter(bigram_list_flat)

Counter({('красный', 'мешочек'): 5,
         ('красный', 'огонь'): 1,
         ('красный', ','): 2,
         ('красный', 'платок'): 1,
         ('красный', '.'): 1})

# Задание 4
Расчитайте ненормализованное растояние Дамерау-Левенштейна и ненормализованное растояние Левенштейна между этими словами (каждое с каждым), найдите пару слов, для которых эти расстояния отличаются. Вставьте эти слова в форму

In [56]:
words = ["решение","ршеение","ренешик","рещиние","ришение"]

In [58]:
import textdistance

In [61]:
def distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n, m)) space
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1)  # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [62]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.similarity(text, word) 
    
    return similarities.most_common(topn)

In [63]:
distance("решение", "ршеение")

2

In [64]:
distance("решение", "ренешик")

3

In [65]:
distance("решение", "рещиние")

2

In [66]:
distance("решение", "ришение")

1

# Задание 5
Загрузите токенизатор distilbert-base-uncased из huggingface. Токенизируйте предложения из Анны Карениной с помощью предобученного токенизатора и обучите Fastext модель со следующими параметрами: cbow, размер вектора 300, размер окна 5, минимальный размер символьного нграмма - 2, максимальный размер символьного нграмма - 7. Найдите ближайшее слово к слову "каренина", вставьте его в форму.

In [75]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 321.9 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [59 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-3.8
      creating build/lib.macosx-10.9-x86_64-3.8/fasttext
      copying python/fasttext_module/fasttext/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/fasttext
      copying python/fasttext_module/fasttext/FastText.py -> build/lib.macosx-10.9-x86_64-3.8/fasttext
      creating build/lib.macosx-10.9-x86_64-3.8/fasttext/util
      copying python/fasttext_module/fasttext/util/util.py -> build/lib.macosx-10.9-x86_64-3.8/fasttext/util
      copying python/fasttext_module/fasttext/util/__init__.py -> build/lib.macosx-10.9-x86_64

In [86]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")



all_tokens = [tokenizer.tokenize(sent) for sent in sentences] 

from gensim.models import FastText

Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors


In [87]:
model = FastText(sentences=all_tokens, vector_size=300, window=5, min_n=2, max_n=7)

model.wv.most_similar('каренина')

[('##ка', 0.5655199289321899),
 ('а', 0.5042127370834351),
 ('##на', 0.35036689043045044),
 ('##ович', 0.3317340612411499),
 ('landau', 0.3216838538646698),
 ('–', 0.2716900706291199),
 ('com', 0.2664029002189636),
 ('pardon', 0.2625886797904968),
 ('tan', 0.24821031093597412),
 ('cher', 0.24624980986118317)]